In [46]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.utils import save_image
from glob import glob

In [49]:
class VoiceMFCC(Dataset):
    """Voice MFCC spectra dataset."""

    def __init__(self, csv_files, standardize=False):
        """
        Preconditions: csv files must contain matrices of the same dimension
        Args:
            csv_files (string or list): list of filenames/pathnames of csv files 
                                        with MFCC spectrogram matrices
            transform (callable, optional): Optional transform to be applied
                                            on a sample.
        """
        # ensure csv_files is a list
        if type(csv_files) == str:
            csv_files = [csv_files]
        
        # load csv files with the MFCC spectra into a 3D tensor
        matrices = []
        for f in csv_files:
            matrix = np.loadtxt(f, delimiter=',', dtype=np.float32)
            if standardize:
                matrix = (matrix - np.mean(matrix)) / np.std(matrix)
            matrices.append(matrix)
        self.X = torch.Tensor(matrices)
        
    def __len__(self):
        return len(self.X.shape[0])

    def __getitem__(self, idx):
        sample = self.X[idx]

        return sample 

In [50]:
csv_files = glob("data/train_mfcc/*.csv")
dataset = VoiceMFCC(csv_files, standardize=True)

In [ ]:
# split dataset here

In [ ]:
BATCH_SIZE = 20
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# HS TO DO
num_epochs = 10
batch_size = 128
learning_rate = 1e-3

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
# model = autoencoder().cuda()
model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [121]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
#         img = Variable(img)#.cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.data.item())) # EDIT: changed [0] to .item()
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './dc_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './conv_autoencoder.pth')

epoch [1/10], loss:0.3421
epoch [2/10], loss:0.2038
epoch [3/10], loss:0.1631
epoch [4/10], loss:0.1292
epoch [5/10], loss:0.1311
epoch [6/10], loss:0.1281
epoch [7/10], loss:0.1196
epoch [8/10], loss:0.1110
epoch [9/10], loss:0.1068
epoch [10/10], loss:0.1221


In [122]:
img, _ = dataset[0]
img = img.view((1,1,28,28))
# img = Variable(img)#.cuda()
output = model(img)

In [123]:
# input
transforms.ToPILImage()(img.view(28,28))

In [124]:
# output
transforms.ToPILImage()(output[0].view(28,28))